In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#import json
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
#import seaborn as sns

import keras_tuner
import tensorflow as tf
import tensorflow.keras.layers as tfl

from sklearn.metrics import confusion_matrix, classification_report

2023-01-06 14:33:40.282054: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
pd.options.display.max_seq_items = 2000
pd.options.display.max_columns = 500
pd.options.display.max_rows = 800
pd.options.display.max_colwidth = 200
pd.options.display.width = 800

In [4]:
base_url = '/Users/francesco/REPOS/nam-nat-CNN-clf/app'
os.chdir(base_url)

In [5]:
from models.hypermodel import hyperband_tuner, model

### Load artifacts

In [6]:
with np.load(os.path.join(base_url, "artifacts/training_data", "nationality_data.npz"), allow_pickle=True) as f:
    x_train, y_train = f["x_train"], f["y_train"]
    x_val, y_val = f["x_val"], f["y_val"]
    
print(x_train.shape)
print(x_val.shape)

(3990, 16)
(705, 16)


In [7]:
with open(os.path.join(base_url, "artifacts/training_data", "nationality_tokenizer.pkl"), "rb") as f:
    tokenizer = pickle.load(f)

### Build model

In [8]:
# Params
embedding_size = len(tokenizer.word_index)
print("embedding_size = ", embedding_size)
metrics = [tf.keras.metrics.CategoricalAccuracy()]
print("metrics = ", metrics)
softmax_units = y_train.shape[1]
print("softmax_units = ", softmax_units)
input_shape = x_train.shape[1]
print("input_shape = ", input_shape)

embedding_size =  33
metrics =  [<keras.metrics.metrics.CategoricalAccuracy object at 0x7fcb0f444eb0>]
softmax_units =  7
input_shape =  16


2023-01-06 14:33:46.167122: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
# Batch data
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(64)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(64)

### Single model

In [49]:
my_model = model(input_shape, metrics, softmax_units, embedding_size)

16


In [50]:
my_model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 16, 33)            1122      
                                                                 
 conv1d_14 (Conv1D)          (None, 10, 16)            3712      
                                                                 
 conv1d_15 (Conv1D)          (None, 6, 16)             1296      
                                                                 
 conv1d_16 (Conv1D)          (None, 4, 16)             784       
                                                                 
 flatten_9 (Flatten)         (None, 64)                0         
                                                                 
 dense_20 (Dense)            (None, 64)                4160      
                                                                 
 dense_21 (Dense)            (None, 128)             

In [51]:
history=my_model.fit(x_train, y_train, epochs=30, batch_size=64,verbose = 1,validation_data=(x_val,y_val))

Epoch 1/30
63/63 [==============================] - 3s 12ms/step - loss: 1.6622 - categorical_accuracy: 0.4260 - val_loss: 1.4285 - val_categorical_accuracy: 0.4738
Epoch 2/30
63/63 [==============================] - 0s 7ms/step - loss: 1.3281 - categorical_accuracy: 0.4990 - val_loss: 1.2763 - val_categorical_accuracy: 0.5362
Epoch 3/30
63/63 [==============================] - 0s 6ms/step - loss: 1.2454 - categorical_accuracy: 0.5336 - val_loss: 1.2385 - val_categorical_accuracy: 0.5504
Epoch 4/30
63/63 [==============================] - 0s 7ms/step - loss: 1.1844 - categorical_accuracy: 0.5559 - val_loss: 1.2348 - val_categorical_accuracy: 0.5589
Epoch 5/30
63/63 [==============================] - 1s 11ms/step - loss: 1.1517 - categorical_accuracy: 0.5684 - val_loss: 1.1909 - val_categorical_accuracy: 0.5702
Epoch 6/30
63/63 [==============================] - 1s 15ms/step - loss: 1.1310 - categorical_accuracy: 0.5779 - val_loss: 1.1744 - val_categorical_accuracy: 0.5759
Epoch 7/30
63

In [18]:
project_name = "tuning_nationality"
directory = "logs/nat_hpt"
tuner = hyperband_tuner(input_shape, metrics, softmax_units, embedding_size, project_name, directory)

In [19]:
tuner.search_space_summary(extended=True)

Search space summary
Default search space size: 5
dropout_1 (Float)
{'default': 0.3, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': None}
dense_units_1 (Choice)
{'default': 64, 'conditions': [], 'values': [64, 128, 256], 'ordered': True}
dropout_2 (Float)
{'default': 0.5, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': None}
dense_units_2 (Choice)
{'default': 64, 'conditions': [], 'values': [64, 128, 256], 'ordered': True}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


### Tuning

In [20]:
hpt_dir = "logs/tb_nat_hpt"
fit_dir = "logs/tb_nat_fit"
print(hpt_dir)
tensorboard_callback_htp = tf.keras.callbacks.TensorBoard(log_dir=hpt_dir, histogram_freq=1)
tensorboard_callback_fit = tf.keras.callbacks.TensorBoard(log_dir=fit_dir, histogram_freq=1)

logs/tb_nat_hpt


In [21]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [22]:
# Estimated trials simulation:
sim_max_epochs = 10
sim_iterations = 1
sim_factor = 3
est_trials = sim_iterations*sim_max_epochs*(math.log(sim_max_epochs, sim_factor))**2
print("Estimated trials: ", est_trials)

Estimated trials:  43.92810535176964


In [23]:
tuner.search(
    train_dataset,
    validation_data=val_dataset,
    callbacks=[tensorboard_callback_htp, tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)]
)

Trial 90 Complete [00h 00m 35s]
val_loss: 1.0847415924072266

Best val_loss So Far: 1.0360156297683716
Total elapsed time: 00h 19m 06s
INFO:tensorflow:Oracle triggered exit


In [24]:
tuner.results_summary(3)

Results summary
Results in logs/nat_hpt/tuning_nationality
Showing 3 best trials
Trial summary
Hyperparameters:
dropout_1: 0.2
dense_units_1: 64
dropout_2: 0.30000000000000004
dense_units_2: 256
lr: 0.006284284551992185
tuner/epochs: 30
tuner/initial_epoch: 10
tuner/bracket: 3
tuner/round: 3
tuner/trial_id: 0049
Score: 1.0360156297683716
Trial summary
Hyperparameters:
dropout_1: 0.2
dense_units_1: 128
dropout_2: 0.5
dense_units_2: 64
lr: 0.004080390057565442
tuner/epochs: 30
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 1.0475938320159912
Trial summary
Hyperparameters:
dropout_1: 0.2
dense_units_1: 64
dropout_2: 0.4
dense_units_2: 256
lr: 0.0021997896716484436
tuner/epochs: 30
tuner/initial_epoch: 10
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0067
Score: 1.0526775121688843


In [ ]:
%tensorboard --logdir $hpt_dir

In [25]:
best_hps = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hps)
print(best_model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 16, 33)            1122      
                                                                 
 conv1d_1 (Conv1D)           (None, 10, 16)            3712      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 5, 16)            0         
 1D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 5, 16)             0         
                                                                 
 lstm_1 (LSTM)               (None, 5, 8)              800       
                                                                 
 flatten_1 (Flatten)         (None, 40)                0         
                                                      